<a href="https://colab.research.google.com/github/reagenhuskey/cs290/blob/main/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
X = pd.read_csv("https://github.com/reagenhuskey/cs290/raw/main/2022%20US%20College%20Rankings.csv")

In [ ]:
X['College Name'] = X['College Name'].str.replace('-', '')

In [ ]:
X.columns

Index(['College Name', 'Adjusted Rank', 'Tuition', 'Enrollment Numbers'], dtype='object')

In [ ]:
affordability = []
cheap = 15000 #consider these are mostly private/really good schools, so their tuition is way more expensive than the average state school.
affordable = 30000
expensive = 100000


In [ ]:
def categorize_t(t):
  if t <= cheap:
      affordability.append('cheap')
      return 'cheap'
  elif t <= affordable:
      affordability.append('affordable')
      return 'affordable'
  elif t <= expensive:
      affordability.append('expensive')
      return 'expensive'

y = pd.Series(affordability)
print(y)
X["Tuition Category"] = X["Tuition"].apply(categorize_t)


0       expensive
1       expensive
2       expensive
3       expensive
4       expensive
          ...    
156     expensive
157    affordable
158    affordable
159    affordable
160    affordable
Length: 161, dtype: object


**unseen instance**

In [ ]:
new_instance = {
    "College Name": "Sewanee",
    "Adjusted Rank": 100, #sounds good enough
    "Tuition Category": "expensive"
}

In [ ]:
def find_prior(y):
  classes = y.unique()
  prior = {}
  for z in classes:
    prior[z] = len(y[y == z]) / len(y)
  return prior

prior = find_prior(y)
print(prior)

{'expensive': 0.8571428571428571, 'affordable': 0.13664596273291926, 'cheap': 0.006211180124223602}


In [ ]:
def find_likelihood(x, y):
  likelihood = {}
  for f in X.columns:
    likelihood[f] = {}
    for c in y.unique():
      f_given_class = X[y == c][f].value_counts(normalize=True).to_dict()
      likelihood[f][c] = f_given_class
  return likelihood



**Naive Bayes Classifier**

In [ ]:
def nbc(X, prior, likelihood, new_instance):
  classes = prior.keys()
  posteriors = {}

  for c in classes:
    posteriors[c] = prior[c]
    for f, value in new_instance.items():
      if value in likelihood[f][c]:
        posteriors[c] *= likelihood[f][c][value]
      else:
        posteriors[c] *= 1e-6
  return max(posteriors, key=posteriors.get)

In [ ]:
likelihood = find_likelihood(X, y)


In [ ]:
predicted_class = nbc(X, prior, likelihood, new_instance)
print(f'Predicted Class for the new instance: {predicted_class}')

Predicted Class for the new instance: expensive


# **Decision Tree Regressor**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
train_set, test_set = train_test_split(X, test_size=0.2)

In [ ]:
X.columns

Index(['College Name', 'Adjusted Rank', 'Tuition', 'Enrollment Numbers',
       'Tuition Category'],
      dtype='object')

In [ ]:
train_set = train_set[['Adjusted Rank', 'Tuition', 'Enrollment Numbers']]
test_set = test_set[['Adjusted Rank', 'Tuition', 'Enrollment Numbers']]
X_train = train_set[['Adjusted Rank', 'Enrollment Numbers']]
y_train = train_set[['Tuition']]

X_test = train_set[['Adjusted Rank', 'Enrollment Numbers']]
y_test = train_set[['Tuition']]




In [ ]:
dt_regressor = DecisionTreeRegressor(random_state=42)

In [ ]:
dt_regressor.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [ ]:
y_prediction = dt_regressor.predict(X_test)

In [ ]:
print(y_prediction[1])

56966.0
